In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [23]:

file_path = '/Users/candice/Downloads/denovo_group.txt'


# simulans

In [24]:
gene_dict = {}
result_list = []

with open(file_path, 'r') as file:
    for line in file:
        # 去除行尾的换行符
        line = line.strip()
        # 按空格分割行
        genes = line.split()
        # 取出第一个基因(dmel基因)
        dmel_gene = genes[0].split('|')[1]
        # 取出其他物种的基因并过滤和处理只保留特定信息
        for gene in genes[1:]:
            if gene.startswith('dsim|Dsimw501_'):
                # 只保留GD部分
                gene_name = gene.split('_')[1]
                result_list.append(f"{dmel_gene}\t{gene_name}\t")
            elif gene.startswith('dsim|NC_'):
                # 提取位置和方向信息
                parts = gene.split('|')[1].split(':')
                position = f"{parts[1]}:{parts[2]}"
                strand = parts[0][-1]  # 提取正负链信息
                result_list.append(f"{dmel_gene}\t{position}\t{strand}")

output_file_path = '/Users/candice/Downloads/dmel_to_dsim_orthologous.txt'
with open(output_file_path, 'w') as output_file:
    for result in result_list:
        output_file.write(result + "\n")

for result in result_list:
    print(result)


FBgn0262810	GD28905	
FBgn0025616	GD16457	
FBgn0040343	GD16454	
FBgn0040343	641498:642320	-
FBgn0040892	GD28135	
FBgn0029539	GD18737	
FBgn0029568	GD16548	
FBgn0040357	GD16549	
FBgn0052811	GD27532	
FBgn0052806	GD24727	
FBgn0052806	1541897:1542539	+
FBgn0029589	GD16412	
FBgn0029589	GD16413	
FBgn0029596	GD24616	
FBgn0025627	GD24730	
FBgn0040353	GD16350	
FBgn0029669	GD16645	
FBgn0029694	GD28270	
FBgn0029694	GD18762	
FBgn0262980	GD16674	
FBgn0262608	GD16678	
FBgn0262609	GD27796	
FBgn0029707	GD16689	
FBgn0029725	4352928:4354994	-
FBgn0029766	GD16732	
FBgn0029766	4969712:4970074	+
FBgn0262611	5388345:5389434	-
FBgn0053664	GD28268	
FBgn0265362	GD24587	
FBgn0265362	6172468:6173004	-
FBgn0261849	GD26926	
FBgn0261615	GD27898	
FBgn0261616	GD27600	
FBgn0052719	GD16152	
FBgn0262612	GD16850	
FBgn0029987	GD16873	
FBgn0030033	GD16894	
FBgn0262976	GD24572	
FBgn0052712	GD24571	
FBgn0262900	GD28459	
FBgn0266050	GD28240	
FBgn0030195	GD28034	
FBgn0052681	GD16036	
FBgn0030290	GD17012	
FBgn0030296	GD24791	
FBg

In [25]:
location = pd.read_csv('/Users/candice/Downloads/dsim-genome.bed', sep='\t', header=None,
                       names=['chrom','start', 'end', 'name','score','strand'])
location['name'] = location['name'].apply(lambda x: x.split('_')[1] if x.startswith('Dsimw501_') else x)
location

,chrom,start,end,name,score,strand
0,NT_479533.1,4304,12624,GD22996,0,-
1,NT_479533.1,12403,17072,GD22995,0,-
2,NT_479533.1,17164,47089,GD22994,0,-
3,NT_479533.1,31513,38933,GD28150,0,-
4,NT_479533.1,52876,57228,GD22997,0,+
...,...,...,...,...,...,...
15087,NC_005781.1,8264,9602,ND4,0,-
15088,NC_005781.1,9603,9892,ND4L,0,-
15089,NC_005781.1,10028,10552,ND6,0,+
15090,NC_005781.1,10556,11692,CYTB,0,+


In [26]:
orthologous = pd.read_csv('/Users/candice/Downloads/dmel_to_dsim_orthologous.txt', sep='\t', header=None,
                          names = ['dmel_gene', 'dsim_gene', 'strand'])
orthologous

,dmel_gene,dsim_gene,strand
0,FBgn0262810,GD28905,NaN
1,FBgn0025616,GD16457,NaN
2,FBgn0040343,GD16454,NaN
3,FBgn0040343,641498:642320,-
4,FBgn0040892,GD28135,NaN
...,...,...,...
503,FBgn0261704,GD28075,NaN
504,FBgn0266259,GD29086,NaN
505,FBgn0039792,GD16925,NaN
506,FBgn0039811,GD16730,NaN


In [27]:

merged_df = pd.merge(orthologous, location, left_on='dsim_gene', right_on='name', how='left')
merged_df
merged_df['strand'] = merged_df.apply(
    lambda row: row['strand_x'] if pd.notnull(row['strand_x']) else row['strand_y'],
    axis=1)
# Split the dsim_gene column where needed and create new columns 'start' and 'end'
split_columns = merged_df['dsim_gene'].str.split(':', expand=True)

# Ensure numeric conversion only happens where appropriate
merged_df['start'] = pd.to_numeric(split_columns[0], errors='coerce').fillna(merged_df['start'])
merged_df['end'] = pd.to_numeric(split_columns[1], errors='coerce').fillna(merged_df['end'])
merged_df = merged_df.drop(columns=['strand_x', 'strand_y', 'dsim_gene', 'score'])

merged_df.to_csv('/Users/candice/Downloads/dmel_to_dsim_ortho_location.csv', index=False, sep='\t')
merged_df


,dmel_gene,chrom,start,end,name,strand
0,FBgn0262810,NC_029795.1,172961.0,173547.0,GD28905,+
1,FBgn0025616,NC_029795.1,610016.0,611702.0,GD16457,-
2,FBgn0040343,NC_029795.1,641542.0,642230.0,GD16454,-
3,FBgn0040343,NaN,641498.0,642320.0,NaN,-
4,FBgn0040892,NC_029795.1,656358.0,665290.0,GD28135,+
...,...,...,...,...,...,...
503,FBgn0261704,NT_479536.1,25563467.0,25563938.0,GD28075,-
504,FBgn0266259,NT_479536.1,25764313.0,25765368.0,GD29086,-
505,FBgn0039792,NT_479536.1,25773072.0,25774784.0,GD16925,-
506,FBgn0039811,NT_479536.1,26073692.0,26074205.0,GD16730,-


# yakuba

In [28]:
gene_dict = {}
result_list = []

with open(file_path, 'r') as file:
    for line in file:
        # 去除行尾的换行符
        line = line.strip()
        # 按空格分割行
        genes = line.split()
        # 取出第一个基因(dmel基因)
        dmel_gene = genes[0].split('|')[1]
        # 取出其他物种的基因并过滤和处理只保留特定信息
        for gene in genes[1:]:
            if gene.startswith('dyak|Dyak_'):
                # 只保留GE部分
                gene_name = gene.split('_')[1]
                result_list.append(f"{dmel_gene}\t{gene_name}\t")
            elif gene.startswith('dyak|NC_'):
                # 提取位置和方向信息
                parts = gene.split('|')[1].split(':')
                position = f"{parts[1]}:{parts[2]}"
                strand = parts[0][-1]  # 提取正负链信息
                result_list.append(f"{dmel_gene}\t{position}\t{strand}")

output_file_path = '/Users/candice/Downloads/dmel_to_dyak_orthologous.txt'
with open(output_file_path, 'w') as output_file:
    for result in result_list:
        output_file.write(result + "\n")

for result in result_list:
    print(result)

FBgn0262810	GE27764	
FBgn0025616	GE29127	
FBgn0040343	GE16563	
FBgn0040343	656066:656938	-
FBgn0040892	GE16644	
FBgn0029539	GE16646	
FBgn0029568	GE16671	
FBgn0040357	GE16672	
FBgn0052811	GE28504	
FBgn0052806	GE16708	
FBgn0029589	GE16510	
FBgn0029596	GE16983	
FBgn0025627	GE16230	
FBgn0040353	GE16915	
FBgn0029669	GE16303	
FBgn0029694	GE16328	
FBgn0029694	GE16329	
FBgn0029694	GE26103	
FBgn0262980	GE27921	
FBgn0262608	GE16336	
FBgn0262609	GE27634	
FBgn0029707	GE16343	
FBgn0029725	3773998:3774507	+
FBgn0029766	GE16406	
FBgn0029766	3143510:3146500	-
FBgn0262611	GE16781	
FBgn0053664	2526118:2526131	-
FBgn0265362	GE16454	
FBgn0265362	2544533:2545029	-
FBgn0261849	GE28294	
FBgn0261615	GE28498	
FBgn0261616	GE28444	
FBgn0052719	GE17520	
FBgn0262612	GE15724	
FBgn0029987	GE15752	
FBgn0030033	GE15778	
FBgn0030033	13031023:13034474	-
FBgn0262976	GE17458	
FBgn0052712	12987157:12991601	+
FBgn0052712	10713254:10713968	-
FBgn0262900	GE15811	
FBgn0266050	17920986:17930884	-
FBgn0030195	GE29149	
FBgn005268

In [29]:
location = pd.read_csv('/Users/candice/Downloads/dyak-genome.bed', sep='\t', header=None,
                       names=['chrom','start', 'end', 'name','score','strand'])
location['name'] = location['name'].apply(lambda x: x.split('_')[1] if x.startswith('Dyak_') else x)
location

,chrom,start,end,name,score,strand
0,NC_011091.1,15336,27108,GE16598,0,+
1,NC_011091.1,27959,77820,GE16599,0,+
2,NC_011091.1,31077,32533,GE16597,0,-
3,NC_011091.1,32560,34026,GE27611,0,-
4,NC_011091.1,63383,64938,GE28906,0,+
...,...,...,...,...,...,...
15932,NC_001322.1,8221,9559,ND4,0,-
15933,NC_001322.1,9559,9849,ND4L,0,-
15934,NC_001322.1,9987,10511,ND6,0,+
15935,NC_001322.1,10515,11651,CYTB,0,+


In [30]:
orthologous = pd.read_csv('/Users/candice/Downloads/dmel_to_dyak_orthologous.txt', sep='\t', header=None,
                          names = ['dmel_gene', 'dyak_gene', 'strand'])
orthologous

,dmel_gene,dyak_gene,strand
0,FBgn0262810,GE27764,NaN
1,FBgn0025616,GE29127,NaN
2,FBgn0040343,GE16563,NaN
3,FBgn0040343,656066:656938,-
4,FBgn0040892,GE16644,NaN
...,...,...,...
479,FBgn0263402,GE28778,NaN
480,FBgn0261704,GE27635,NaN
481,FBgn0039792,GE23372,NaN
482,FBgn0039811,GE23349,NaN


In [31]:

merged_df = pd.merge(orthologous, location, left_on='dyak_gene', right_on='name', how='left')
merged_df
merged_df['strand'] = merged_df.apply(
    lambda row: row['strand_x'] if pd.notnull(row['strand_x']) else row['strand_y'],
    axis=1)
# Split the dsim_gene column where needed and create new columns 'start' and 'end'
split_columns = merged_df['dyak_gene'].str.split(':', expand=True)

# Ensure numeric conversion only happens where appropriate
merged_df['start'] = pd.to_numeric(split_columns[0], errors='coerce').fillna(merged_df['start'])
merged_df['end'] = pd.to_numeric(split_columns[1], errors='coerce').fillna(merged_df['end'])
merged_df = merged_df.drop(columns=['strand_x', 'strand_y', 'dyak_gene', 'score'])

merged_df.to_csv('/Users/candice/Downloads/dmel_to_dyak_ortho_location.csv', index=False, sep='\t')
merged_df


,dmel_gene,chrom,start,end,name,strand
0,FBgn0262810,NC_011091.1,175271.0,175662.0,GE27764,+
1,FBgn0025616,NW_002057603.1,162.0,1503.0,GE29127,+
2,FBgn0040343,NC_011091.1,656258.0,656828.0,GE16563,-
3,FBgn0040343,NaN,656066.0,656938.0,NaN,-
4,FBgn0040892,NC_011091.1,673717.0,674335.0,GE16644,+
...,...,...,...,...,...,...
479,FBgn0263402,NT_167065.1,26997261.0,26997684.0,GE28778,-
480,FBgn0261704,NT_167065.1,27162514.0,27162958.0,GE27635,-
481,FBgn0039792,NT_167065.1,27374640.0,27376331.0,GE23372,-
482,FBgn0039811,NT_167065.1,27689013.0,27689603.0,GE23349,-
